In [3]:
import scipy.io as io
# read data/zuco/task1/Matlab_files/resultsZAB_SR.mat
sentence_data = io.loadmat('data/zuco/task1/Matlab_files/resultsZAB_SR.mat', squeeze_me=True, struct_as_record=False,)['sentenceData']

In [6]:
import numpy as np
import pandas as pd
sn_data = sentence_data[0]
dur = np.array(sn_data.allFixations.duration)
x = np.array(sn_data.allFixations.x)
y = np.array(sn_data.allFixations.y)


In [12]:
sn_data.content

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [ ]:
word_boundary = sn_data.wordbounds
df = pd.DataFrame([], columns=['sn', 'wn', 'fl', 'dur'])
for fix_idx in range(x.shape[0]):
    word_idx = np.where(
        (word_boundary[:, 0] <= x[fix_idx]) &
        (word_boundary[:, 2] >= x[fix_idx]) &
        (word_boundary[:, 1] <= y[fix_idx]) &
        (word_boundary[:, 3] >= y[fix_idx]),
    )
    assert len(word_idx) == 1, 'more than 1 word is matched!'
    # skip fixations outside all boundary boxes
    if word_idx[0].size == 0:
        continue
    fl = ((x[fix_idx] - word_boundary[word_idx, 0]) /
            (word_boundary[word_idx, 2] - word_boundary[word_idx, 0]))[0][0]
    df_tmp = pd.DataFrame(
        [[
            1,  word_idx[0][0] + 1,
            fl, dur[fix_idx],
        ]], columns=['sn', 'wn', 'fl', 'dur'],
    )
    df = pd.concat([df, df_tmp])


In [11]:
words = pd.read_csv("data/zuco/task1/Matlab_files/scanpath.csv", sep='\t')
# check is there is nan in words
words.isnull().values.any()

# check if there is nan or '' in column 'WORD'
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'].isnull().values]
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'] == '']

,id,sn,nw,wn,fl,dur,CURRENT_FIX_INTEREST_AREA_LABEL


In [5]:
import pandas as pd
path = "data/zuco/task2/Matlab_files/scanpath.csv"
df = pd.read_csv(path, sep='\t')
df['id'].unique()


array(['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH',
       'ZKW', 'ZMG', 'ZPH'], dtype=object)

In [30]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv("data/zuco/task2/fixation.csv", sep=',')
sentences = df['sn'].unique()
df_avg = pd.DataFrame([], columns=['sn', 'list_dur'])

for sn in sentences:
    df_tmp = df[df['sn'] == sn]
    # Convert string representations to actual lists
    df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
    
    # Calculate the average fixation duration
    avg_dur = np.nanmean(df_tmp['list_dur'].values.tolist(), axis=0)

    df_avg = pd.concat([df_avg, pd.DataFrame({'sn': [sn], 'list_dur': [avg_dur.tolist()]})], ignore_index=True)

df_avg.to_csv("data/zuco/task2/fixation_avg.csv", index=False)

    


/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to 

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]

df = pd.DataFrame(columns=['list'])
df['list'] = [a]
df['list'].append({'list': b})
df



In [8]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
text = "Replace me by any text you'd like."
input_tokens = tokenizer(text)['input_ids']
attention_ids = tokenizer(text)['attention_mask']
input_tokens = torch.tensor(input_tokens).unsqueeze(0)
attention_ids = torch.tensor(attention_ids).unsqueeze(0)
torch.enable_grad()
model.eval()
outputs = model(input_tokens, attention_ids)
A = model(input_tokens, attention_mask=attention_ids)
A.logits.shape



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


12


torch.Size([1, 12, 30522])